# Lab 7: Configuración de un Application Load Balancer con Dos Instancias en la VPC por Defecto

Este laboratorio te mostrará cómo crear un **Application Load Balancer (ALB)** y lanzar dos instancias EC2 en la **VPC por defecto**. Verás cómo el AB distribuye el tráfico entre ambas instancias, demostrando balanceo de carga y alta disponibilidad.

---

**Objetivos del Laboratorio**

1. Utilizar la VPC por defecto y sus subnets para implementar un ALB.
2. Lanzar dos instancias EC2 en subnets públicas y configurarlas con un servidor web simple.
3. Configurar el ALB para balancear tráfico HTTP entre estas instancias.
4. Verificar el funcionamiento del balanceo al observar las respuestas alternadas entre las dos instancias.

---

**Pasos del Laboratorio**

**1. Identificar la VPC por Defecto y Subnets**

1. Inicia sesión en la **Consola de AWS**: [https://console.aws.amazon.com](https://console.aws.amazon.com/)
2. Ve a **VPC**.
3. En "Your VPCs", identifica la VPC por defecto.
4. En "Subnets", revisa las subnets por defecto. Debes tener al menos dos subnets en diferentes Zonas de Disponibilidad. Anota sus nombres e IDs.

*(En este lab usarás la VPC y subnets por defecto, que ya tienen rutas a Internet.)*

**2. Lanzar Dos Instancias EC2**

1. Ve a **EC2 > Launch Instances**.
2. **Instancia 1**:
    - Nombre: `Instance-1`
    - AMI: Amazon Linux 2
    - Tipo: t2.micro
    - VPC: Por defecto
    - Subnet: Selecciona una subnet pública (por defecto suelen serlo)
    - Security Group: Crea uno nuevo:
        - Permite HTTP (TCP/80) desde `0.0.0.0/0`
        - Permite SSH (TCP/22) desde `My IP` (opcional)
    - User Data:
        
        ```bash
        
        #!/bin/bash
        sudo yum install -y httpd
        sudo systemctl start httpd
        echo "Bienvenido a Instance-1" > /var/www/html/index.html
        
        ```
        
    
    Lanza la instancia.
    
3. **Instancia 2**:
    - Nombre: `Instance-2`
    - MISMA AMI, tipo de instancia, VPC
    - Subnet: Selecciona otra subnet en una zona de disponibilidad distinta.
    - Usa el mismo Security Group.
    - User Data:
        
        ```bash
        
        #!/bin/bash
        sudo yum install -y httpd
        sudo systemctl start httpd
        echo "Bienvenido a Instance-2" > /var/www/html/index.html
        
        ```
        
    
    Lanza la segunda instancia.
    

Espera a que ambas estén "running" y pasen los "Status Checks".

**3. Crear el Application Load Balancer**

1. En la consola de EC2, ve a **Load Balancers** y haz clic en **Create Load Balancer**.
2. Selecciona **Application Load Balancer**.
3. Configuración del ALB:
    - Name: `ALB-Demo`
    - Scheme: Internet-facing
    - Listeners: HTTP (80)
    - VPC: la VPC por defecto
    - Subnets: Selecciona las dos subnets que anotaste antes.
4. Security Group del ALB: Crea uno que permita HTTP (80) desde `0.0.0.0/0`.

Clic en "Next: Configure Routing".

**4. Crear el Target Group**

1. Nombre: `TG-ALB-Demo`
2. Target type: Instances
3. Protocol: HTTP, Port: 80
4. Health checks: Path `/`

Clic en "Next".

**Registrar Targets:**

- Selecciona `Instance-1` y `Instance-2`.
- "Include as pending".
- "Create Target Group".

Regresa a la pantalla del ALB, selecciona `TG-ALB-Demo` y haz clic en "Create Load Balancer".

**5. Verificar el Balanceo**

1. En **Load Balancers**, espera a que el ALB esté en estado "active".
2. Copia el **DNS Name** del ALB.
3. Pega la URL en tu navegador:Verás "Bienvenido a Instance-1" o "Bienvenido a Instance-2".
4. Refresca la página varias veces:La respuesta se alterna entre Instance-1 y Instance-2, demostrando el balanceo de carga.

---

**Resultados Esperados**

- El ALB distribuye el tráfico entre las dos instancias.
- Al refrescar el navegador, verás las respuestas alternadas ("Bienvenido a Instance-1" y "Bienvenido a Instance-2").
- Confirmas así la alta disponibilidad y el balanceo proporcionado por el ALB.

---

**Conclusiones**

En este laboratorio configuraste un ALB usando la VPC por defecto y sus subnets públicas, lanzaste dos instancias EC2 con un servidor HTTP simple y verificaste el reparto de tráfico. Esta experiencia es fundamental para entender cómo aumentar la escalabilidad, disponibilidad y resiliencia en aplicaciones web con AWS.